## Задача - 1. Насколько похорошела Москва

### Импорт библиотек и загрузка данных

In [1]:
import os
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [2]:
! head -n 1 data/data-102277-2020-04-02.csv

"ID";"LibraryName";"AdmArea";"District";"Address";"NumberOfAccessPoints";"WiFiName";"CoverageArea";"FunctionFlag";"AccessFlag";"Password";"Latitude_WGS84";"Longitude_WGS84";"global_id";"geoData";


In [3]:
! head -n 1 data/parks.csv

"ID";"global_id";"Name";"AdmArea";"District";"ParkName";"WiFiName";"CoverageArea";"FunctionFlag";"AccessFlag";"Password";"Longitude_WGS84";"Latitude_WGS84";"geoData";


In [4]:
#Загружаем данные (не парки)
df = pd.concat([pd.read_csv('data/'+file, sep=';', encoding='cp1251') for file in os.listdir('data') if 'parks' not in file])
print(df.shape)
df.head(2)

(590, 19)


,AccessFlag,Address,AdmArea,CinemaName,CoverageArea,CulturalCenterName,District,FunctionFlag,ID,Latitude_WGS84,LibraryName,Longitude_WGS84,NumberOfAccessPoints,Password,Unnamed: 0,Unnamed: 15,WiFiName,geoData,global_id
0,открытая сеть,"город Москва, улица Боженко, дом 9",Западный административный округ,NaN,50.0,Государственное бюджетное учреждение культуры ...,район Кунцево,действует,1.0,55.732291,NaN,37.405202,1.0,NaN,NaN,NaN,Moscow_WiFi_Free,"{type=Point, coordinates=[37.405202, 55.732291]}",879812827.0
1,открытая сеть,"город Москва, Полоцкая улица, дом 16",Западный административный округ,NaN,50.0,Государственное бюджетное учреждение культуры ...,район Кунцево,действует,2.0,55.734456,NaN,37.425180,1.0,NaN,NaN,NaN,Moscow_WiFi_Free,"{type=Point, coordinates=[37.42518, 55.734456]}",879812830.0


In [5]:
# Загружаем данные по паркам, там формат другой
df_parks = pd.read_csv('data/parks.csv', sep=';', encoding='cp1251')
print(df_parks.shape)
df_parks.head(2)

(1235, 15)


,ID,global_id,Name,AdmArea,District,ParkName,WiFiName,CoverageArea,FunctionFlag,AccessFlag,Password,Longitude_WGS84,Latitude_WGS84,geoData,Unnamed: 14
0,337,46808732,Точка доступа №337,Юго-Восточный административный округ,район Кузьминки,Государственное автономное учреждение культуры...,Moscow_WiFi_Free,20,действует,открытая сеть,NaN,37.769489,55.69501,"{type=Point, coordinates=[37.769489, 55.69501]}",NaN
1,338,46808734,Точка доступа №338,Юго-Восточный административный округ,район Кузьминки,Государственное автономное учреждение культуры...,Moscow_WiFi_Free,20,действует,открытая сеть,NaN,37.768175,55.695218,"{type=Point, coordinates=[37.768175, 55.695218]}",NaN


### Предобработка

In [6]:
# Отбираем необходимые поля
df = df[['AdmArea', 'District', 'Address', 'global_id']].drop_duplicates()
print(df.shape)
df.head(2)

(590, 4)


,AdmArea,District,Address,global_id
0,Западный административный округ,район Кунцево,"город Москва, улица Боженко, дом 9",879812827.0
1,Западный административный округ,район Кунцево,"город Москва, Полоцкая улица, дом 16",879812830.0


In [7]:
# Пропуски
df.isnull().sum()

AdmArea      0
District     0
Address      0
global_id    0
dtype: int64

In [8]:
# Отбираем необходимые поля
df_parks = df_parks[['Name', 'District', 'ParkName', 'global_id']].drop_duplicates()
print(df_parks.shape)
df_parks.head(2)

(1235, 4)


,Name,District,ParkName,global_id
0,Точка доступа №337,район Кузьминки,Государственное автономное учреждение культуры...,46808732
1,Точка доступа №338,район Кузьминки,Государственное автономное учреждение культуры...,46808734


In [9]:
# Пропуски
df_parks.isnull().sum()

Name         0
District     0
ParkName     0
global_id    0
dtype: int64

In [15]:
# Проверяем, что global_id - идентификатор
len(df['global_id'].unique()) == df.shape[0] , len(df_parks['global_id'].unique()) == df_parks.shape[0]

(True, True)

In [11]:
#Сплитим улицу из шаблона адреса
def split_address(row):
    return row.split(',')[1]

df['street'] = df['Address'].apply(lambda x: split_address(x))
df.head(2)

,AdmArea,District,Address,global_id,street
0,Западный административный округ,район Кунцево,"город Москва, улица Боженко, дом 9",879812827.0,улица Боженко
1,Западный административный округ,район Кунцево,"город Москва, Полоцкая улица, дом 16",879812830.0,Полоцкая улица


### Считаем топ-10 улиц по кол-ву wi-fi точек

In [12]:
print('Кол-во уникальных улиц -', len(df['street'].unique()))

Кол-во уникальных улиц - 415


In [13]:
# Выводим топ-10 улиц с большим кол-вом wi-fi точек
df[['global_id', 'street']].groupby('street')\
                           .agg({'global_id':'count'})\
                           .rename(columns={'global_id':'count'})\
                           .reset_index()\
                           .sort_values('count', ascending=False)\
                           .head(10)

,street,count
272,город Зеленоград,17
82,Дмитровское шоссе,6
284,поселение Рязановское,5
108,Кастанаевская улица,5
64,Боровское шоссе,5
134,Лукинская улица,5
362,улица Маршала Катукова,5
198,Рублёвское шоссе,4
109,Каширское шоссе,4
380,улица Обручева,4


### Считаем топ-10 парков по кол-ву wi-fi точек

In [ ]:
# Отбираем необходимые поля
print('Кол-во уникальных парков -', len(df_parks['ParkName'].unique()))

In [22]:
# Выводим топ-10 парков с большим кол-вом wi-fi точек
pd.set_option('display.max_colwidth', 140)
df_parks[['global_id', 'ParkName']].groupby('ParkName')\
                           .agg({'global_id':'count'})\
                           .rename(columns={'global_id':'count'})\
                           .reset_index()\
                           .sort_values('count', ascending=False)\
                           .head(10)

,ParkName,count
21,Государственное автономное учреждение культуры города Москвы «Парк культуры и отдыха «Сокольники»,165
37,Государственное бюджетное учреждение культуры города Москвы «Московский государственный объединенный художественный историко-архитектурн...,121
8,Государственное автономное учреждение культуры города Москвы «Измайловский Парк культуры и отдыха»,120
1,Государственное автономное учреждение города Москвы «Московский государственный зоологический парк»,74
14,"Государственное автономное учреждение культуры города Москвы «Музейно-парковый комплекс «Северное Тушино», зона отдыха «Левобережье»",51
32,"Государственное автономное учреждение культуры города Москвы «Центральный парк культуры и отдыха имени М.Горького», Парк искусств «Музеон»",51
28,Государственное автономное учреждение культуры города Москвы «Поклонная гора»,50
19,Государственное автономное учреждение культуры города Москвы «Парк культуры и отдыха «Красная Пресня»,49
9,Государственное автономное учреждение культуры города Москвы «Лианозовский парк культуры и отдыха»,49
16,"Государственное автономное учреждение культуры города Москвы «Музейно-парковый комплекс «Северное Тушино», парк «Северное Тушино»",49


## Задача - 2. Статистика оказаний услуг

Есть таблица, в которой записаны услуги пациента в больнице: кто, где, когда, какая услуга. Напишите SQL-запрос, который выведет топ 10 самых популярных услуг с указанием количества оказаний. Пример таблицы ниже:

| WHO  | WHERE  | WHEN   | SERVICE   |
|------|--------|--------|-----------|
|Ivanov| Moscow |1-1-2020| operation |

In [24]:
# Пример запроса
query = """
SELECT TOP (10) SERVICE, COUNT(*) AS CNT_SERVICE
FROM TABLE
GROUP BY SERVICE
ORDER BY CNT_SERVICE DESC
"""

## Задача - 3. Обновление Python

python3 -V у вас выдаёт 3.7, вы хотите обновиться до 3.8. У вас Убунту или Макось. Как вы будете это делать?

In [25]:
! python3 --version

Python 3.7.4


----------

При помощи командной строки <br />
*! brew install python3*

--------

Либо скачать и установить c ресурса <br />
*https://www.python.org/downloads/*